In [1]:
import torch
import pickle

with open( "x_train_hist.p", "rb" ) as file:
    x_train_hist = pickle.load(file)
    
with open( "y_train_hist.p", "rb" ) as file:
    y_train_hist = pickle.load(file)
    
with open( "x_valid_hist.p", "rb" ) as file:
    x_valid_hist = pickle.load(file)
    
with open( "y_valid_hist.p", "rb" ) as file:
    y_valid_hist = pickle.load(file)

In [2]:
import torch
import random
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from tqdm.auto import tqdm

from mriqa_dataset import MRIQADataset
from networks import ClassicCNN, PhilsClassicCnn, CatNet

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Link: https://arxiv.org/abs/2003.04696



In [3]:
import torch_optimizer as optim
num_epochs = len(x_train_hist)

net = CatNet(num_classes=5)
net = net.cuda()

#optimizer = optim.Adam(net.parameters())
optimizer = optim.Ranger(
    net.parameters(),
    lr=9e-4,
    alpha=0.5,
    k=6,
    N_sma_threshhold=5,
    betas=(.95, 0.999),
    eps=1e-5,
    weight_decay=0
)

flat_lr = round(num_epochs * 0.7)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(num_epochs-flat_lr))
#lmbda = lambda epoch: 0.95
#scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)
ce = CrossEntropyLoss().cuda()

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']



print(f"I will do flat Learning rate for {flat_lr} epochs and then do {num_epochs-flat_lr} epochs cosine annealing")

I will do flat Learning rate for 217 epochs and then do 93 epochs cosine annealing


In [4]:
best_val_loss = 999

loss_csv = open('losses.csv', 'w')
loss_csv.write('epoch,training,validation\n')
num_mini_batches = 13
    
print("start training")
for epoch in range(num_epochs):
    epoch_loss = 0.0
    net.train()
    
    train_batches = x_train_hist[epoch]
    train_labels = y_train_hist[epoch]

    # train loop
    for sample, label in tqdm(zip(train_batches, train_labels), total=len(train_batches), leave=False):       
        sample = sample.cuda()
        label = label.cuda()

        prediction = net(sample)
        loss = ce(prediction, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), 1)
        optimizer.step()
        epoch_loss += loss.item()
    
    # do annealing after 70% of the training time
    if epoch > flat_lr:
        scheduler.step()

    print('[{}] train-loss: {}'.format(epoch, epoch_loss / num_mini_batches))
    loss_csv.write(str(epoch) + ',' + str(epoch_loss / num_mini_batches))
    loss_csv.flush()

    # validation loop
    net.eval()
    mean_validation_loss = 0
    num_validation_mini_batches = 11
    
    if epoch % 3 == 0:
        valid_batches = x_valid_hist[epoch//5]
        valid_labels = y_valid_hist[epoch//5]
        with torch.no_grad():
            for sample, label in tqdm(zip(valid_batches, valid_labels), total=len(valid_batches), leave=False):              
                sample = sample.cuda()
                label = label.cuda()

                prediction = net(sample)
                validation_loss = ce(prediction, label)

                mean_validation_loss += validation_loss.item()
            print(f'[{epoch}] validation-loss: {mean_validation_loss / num_validation_mini_batches} Lerning Rate {get_lr(optimizer)}')
            loss_csv.write(',' + str(mean_validation_loss / num_validation_mini_batches) + '\n')
            loss_csv.flush()

        # save best model
        if mean_validation_loss <= best_val_loss:
            torch.save({'epoch': epoch,
                        'model_state_dict': net.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': validation_loss.item()}, 'checkpoint_best')
            best_val_loss = mean_validation_loss

print('DONE.')

start training


..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value)


[0] train-loss: 0.49916683710538423


[0] validation-loss: 0.43635866858742456 Lerning Rate 0.0009


[1] train-loss: 0.5042969080118033


...

[306] validation-loss: 0.2510438453067433 Lerning Rate 4.101809328792519e-06


[307] train-loss: 0.2901087999343872


[308] train-loss: 0.29894136924010056


[309] train-loss: 0.28191441756028396


[309] validation-loss: 0.2511053518815474 Lerning Rate 2.567290816268995e-07
DONE.


In [5]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [6]:
prediction

tensor([[8.1751e-09, 7.7866e-07, 9.1112e-08, 1.0000e+00, 1.1654e-09],
        [7.8917e-09, 7.9131e-07, 1.0044e-07, 1.0000e+00, 1.7672e-09],
        [2.6293e-07, 1.0000e+00, 5.9445e-11, 5.9596e-10, 8.3633e-08],
        [5.4529e-07, 5.5853e-08, 1.0000e+00, 2.9399e-08, 1.1039e-06],
        [2.1192e-08, 2.3380e-12, 1.0000e+00, 4.1937e-11, 5.7587e-11],
        [9.9927e-01, 1.3875e-04, 5.0521e-04, 7.6006e-05, 5.2926e-06],
        [9.2417e-09, 2.5784e-07, 7.7077e-08, 1.0000e+00, 3.5888e-10],
        [4.8748e-12, 1.0000e+00, 9.2828e-12, 9.4536e-10, 1.0514e-09],
        [1.1424e-08, 1.0000e+00, 2.1505e-10, 4.7224e-08, 8.8493e-09],
        [4.8963e-06, 3.7303e-07, 1.4507e-09, 7.5827e-11, 9.9999e-01],
        [8.9398e-06, 3.5119e-09, 9.9999e-01, 3.8491e-08, 2.3634e-08],
        [1.7540e-07, 5.6175e-10, 1.0000e+00, 4.0932e-10, 1.5227e-08],
        [1.0735e-06, 6.0700e-08, 1.9198e-06, 2.2887e-09, 1.0000e+00],
        [8.3475e-07, 1.5465e-08, 1.0000e+00, 2.0683e-08, 8.1588e-07],
        [9.9997e-01,

In [7]:
label

tensor([3, 3, 1, 2, 2, 0, 3, 1, 1, 4, 2, 2, 4, 2, 0, 1, 2, 3, 3, 1, 3],
       device='cuda:0')

In [8]:
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch, labels in tqdm(zip(x_train_hist, y_train_hist)):
        for minibatch, label in zip(batch, labels):
            sample = minibatch.cuda()

            prediction = net(sample)
            all_predictions.append(prediction.cpu())
            
            all_labels.append(label)

In [9]:
all_predictions = [item.argmax().tolist() for sublist in all_predictions for item in sublist]
all_labels = [item.tolist() for sublist in all_labels for item in sublist]
print(classification_report(all_labels, all_predictions))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      6339
           1       1.00      0.96      0.98      6175
           2       0.99      0.99      0.99      6165
           3       1.00      1.00      1.00      6208
           4       0.96      0.99      0.98      6113

    accuracy                           0.99     31000
   macro avg       0.99      0.99      0.99     31000
weighted avg       0.99      0.99      0.99     31000



In [11]:
confusion_matrix(all_labels, all_predictions)

array([[6295,    1,   40,    0,    3],
       [ 185, 5806,   23,    0,  161],
       [   7,    0, 6158,    0,    0],
       [   0,    0,    0, 6208,    0],
       [  41,    7,   32,    0, 6033]], dtype=int64)

In [10]:
confusion_matrix(all_labels, all_predictions)

array([[6283,    3,   20,    0,   33],
       [  52, 5898,    9,    0,  216],
       [  40,    0, 6125,    0,    0],
       [   0,    0,    0, 6208,    0],
       [  28,    2,   13,    0, 6070]], dtype=int64)

In [11]:
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch, labels in tqdm(zip(x_valid_hist, y_valid_hist)):
        for minibatch, label in zip(batch, labels):
            sample = minibatch.cuda()

            prediction = net(sample)
            all_predictions.append(prediction.cpu())
            
            all_labels.append(label)

In [12]:
all_predictions = [item.argmax().tolist() for sublist in all_predictions for item in sublist]
all_labels = [item.tolist() for sublist in all_labels for item in sublist]
print(classification_report(all_labels, all_predictions))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2727
           1       0.99      0.96      0.97      2714
           2       0.99      1.00      0.99      2710
           3       1.00      1.00      1.00      2714
           4       0.98      0.98      0.98      2735

    accuracy                           0.99     13600
   macro avg       0.99      0.99      0.99     13600
weighted avg       0.99      0.99      0.99     13600



In [13]:
print(confusion_matrix(all_labels, all_predictions))

[[2717    0    6    0    4]
 [  45 2605    8    0   56]
 [   2    0 2707    0    1]
 [   0    0    0 2714    0]
 [  10   25   11    0 2689]]


In [10]:
torch.save({'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': validation_loss.item()}, 'final')

In [11]:
model = torch.load("checkpoint_best")

In [12]:
model = CatNet(num_classes=5)
optimizer = optim.Ranger(
    net.parameters(),
    lr=1e-3,
    alpha=0.5,
    k=6,
    N_sma_threshhold=5,
    betas=(.95, 0.999),
    eps=1e-5,
    weight_decay=0
)

checkpoint = torch.load("checkpoint_best")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
model.cuda()

CatNet(
  (block1): Sequential(
    (0): Conv2d(1, 82, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (1): BatchNorm2d(82, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): mish_layer()
  )
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (block2): skip_connection_block(
    (conv_il): Conv2d(82, 82, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (batn_il): BatchNorm2d(82, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_ol): Conv2d(82, 82, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batn_ol): BatchNorm2d(82, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_sl): Conv2d(82, 82, kernel_size=(1, 1), stride=(2, 2))
    (batn_sl): BatchNorm2d(82, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block3): skip_connection_block(
    (conv_il): Conv2d(82, 82, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (batn_il): BatchNor

In [13]:
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch, labels in tqdm(zip(x_valid_hist, y_valid_hist)):
        for minibatch, label in zip(batch, labels):
            sample = minibatch.cuda()

            prediction = model(sample)
            all_predictions.append(prediction.cpu())
            
            all_labels.append(label)

In [14]:
all_predictions = [item.argmax().tolist() for sublist in all_predictions for item in sublist]
all_labels = [item.tolist() for sublist in all_labels for item in sublist]
print(classification_report(all_labels, all_predictions))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2727
           1       1.00      0.97      0.98      2714
           2       0.99      1.00      0.99      2710
           3       1.00      1.00      1.00      2714
           4       0.98      1.00      0.99      2735

    accuracy                           0.99     13600
   macro avg       0.99      0.99      0.99     13600
weighted avg       0.99      0.99      0.99     13600



In [15]:
print(confusion_matrix(all_labels, all_predictions))

[[2698    3   16    0   10]
 [   8 2642   10    1   53]
 [   0    0 2708    0    2]
 [   0    0    0 2714    0]
 [   6    6    0    0 2723]]
